In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
np.random.seed = 42
import category_encoders as ce

In [ ]:
allegro_df = pd.read_csv('allegro-api-transactions.csv')
allegro_df.head()

# Target encoding

In [ ]:
means = allegro_df.groupby('it_location')['price'].mean()
print('Liczba kategorii: ' + str(len(allegro_df['it_location'].unique())))
print(means)

In [ ]:
TG = allegro_df.copy()
TG['it_location'] = allegro_df['it_location'].map(means)
TG.head()

Przewagą nad one-hot encoding jest mniejsza zajętość pamięciowa, nietworzenie dodatkowej ilości kolumn dla każdej kategorii. W tym zadaniu zmienna ma 6291 kategorii, a więc tyle dodatkowych kolumn by powstało. Jego kolejną zaletą jest, że zmienia zmienną kategoryczną na zmienną celu. 

# One-hot endoding

In [ ]:
OHE = pd.concat([
    pd.get_dummies(allegro_df.main_category, prefix='MainCategory'),
    allegro_df], axis=1).drop(['main_category'], axis=1)
print(OHE.head())

# Leave One Out Encoder

In [ ]:
loe = ce.LeaveOneOutEncoder(cols = ['main_category'], return_df = True)
LOE = loe.fit_transform(allegro_df, allegro_df['price'])

print(LOE.head())

# Count Encoding

In [ ]:
pe = ce.CountEncoder(cols = ['main_category'], return_df = True)
PE = pe.fit_transform(allegro_df, allegro_df['price'])

print(PE.head())

Wybrałem *LeaveOneOut Encoding* oraz *CountEncoding*. Pierwszy z nich działa podobnie jak *Target Encoding*, z tą różnicą, że przy wyliczaniu średniej nie bierze pod uwagę danego rekordu. Działa tylko, jeżeli zmienna celu jest typu binarnego albo ciągłego. Natomiast *CountEncoding* zamienia nazwę kategorii na liczbę jej wystąpień, dlatego nie ma żadnych ograniczeń co do typu zmiennej celu.

# Zad 2

In [ ]:
allegro_df_mod = allegro_df.loc[:10000, ['price', 'it_seller_rating', 'it_quantity']]
print(allegro_df_mod)
cols = allegro_df_mod.columns

rmse = [0] * 10
sr = np.mean(allegro_df_mod['it_seller_rating'])
print(sr)

In [ ]:
from sklearn.impute import KNNImputer

remove_n = int(0.1 * len(allegro_df_mod))

for r in range (len(rmse)):
    allegro_tmp = allegro_df_mod.copy()
    r_temp = 0
    drop_indices = np.array(np.random.choice(allegro_df_mod.index, remove_n, replace=False))

    for i in drop_indices:
        allegro_tmp.loc[i, 'it_seller_rating'] = np.NaN

    imputer = KNNImputer(n_neighbors=3, weights="uniform")
    NN = pd.DataFrame(imputer.fit_transform(allegro_tmp), columns = cols)

    for i in range (len(NN)):
        r_temp += (sr - NN.loc[i, 'it_seller_rating'])**2
    r_temp /= len(NN)
    r_temp = np.sqrt(r_temp).round(2)

    rmse[r] = r_temp
print(rmse)
std_r = np.std(rmse)
mean_r = np.mean(rmse)

In [ ]:
rmse_org = 0
for i in range (len(allegro_df_mod)):
    rmse_org += (sr - allegro_df_mod.loc[i, 'it_seller_rating'])**2
rmse_org = rmse_org / len(allegro_df_mod)
rmse_org = np.sqrt(rmse_org).round(2)

rmse_org_IQ = 0
for i in range (len(allegro_df_mod)):
    rmse_org_IQ += (sr - allegro_df_mod.loc[i, 'it_quantity'])**2
rmse_org_IQ = rmse_org_IQ / len(allegro_df_mod)
rmse_org_IQ = np.sqrt(rmse_org_IQ).round(2)

In [ ]:
print(f'Miara RMSE dla oryginalnych wartości: {rmse_org}.')
print(f'Miary RMSE dla losowo usuniętych: {rmse}.')
print(f'Średnia miary RMSE dla losowo usuniętych: {mean_r:.2f}.')
print(f'Odchylenie standardowe miary RMSE dla losowo usuniętych: {std_r:.2f}.')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
plt.plot(rmse, label = 'RMSE po usunięciu danych')
plt.plot(range(10), [rmse_org]*10, label = 'RMSE bez usunięcia danych')
plt.plot(range(10), [mean_r]*10, label = 'Średnia RMSE po usunięciu danych')
ax.set_title('Wizualizacji wartości miary RMSE')
ax.set_ylim([34400, 35700])
ax.legend()
legend = ax.legend(loc='center right', shadow=True, fontsize='medium')
plt.show()

In [ ]:
rmse_SR = [0]*10
rmse_IQ = [0]*10

for r in range (len(rmse)):
    allegro_tmp = allegro_df_mod.copy()
    r_temp_SR = 0
    r_temp_IQ = 0
    
    drop_indices = np.array(np.random.choice(allegro_df_mod.index, remove_n, replace=False))
    for i in drop_indices:
        allegro_tmp.loc[i, 'it_seller_rating'] = np.NaN
        
    drop_indices = np.array(np.random.choice(allegro_df_mod.index, remove_n, replace=False))
    for i in drop_indices:
        allegro_tmp.loc[i, 'it_quantity'] = np.NaN

    imputer = KNNImputer(n_neighbors=3, weights="uniform")
    NN = pd.DataFrame(imputer.fit_transform(allegro_tmp), columns = cols)

    for i in range (len(NN)):
        r_temp_SR += (sr - NN.loc[i, 'it_seller_rating'])**2
        r_temp_IQ += (sr - NN.loc[i, 'it_quantity'])**2
    r_temp_SR /= len(NN)
    r_temp_SR = np.sqrt(r_temp_SR).round(2)
    r_temp_IQ /= len(NN)
    r_temp_IQ = np.sqrt(r_temp_IQ).round(2)

    rmse_SR[r] = r_temp_SR
    rmse_IQ[r] = r_temp_IQ
std_SR = np.std(rmse_SR)
mean_SR = np.mean(rmse_SR)
std_IQ = np.std(rmse_IQ)
mean_IQ = np.mean(rmse_IQ)

mean_SR_org = allegro_df_mod['it_seller_rating'].mean()
mean_IQ_org = allegro_df_mod['it_quantity'].mean()

In [ ]:
x = 'it_seller_rating'
print(f'Miara RMSE dla oryginalnych wartości zmiennej {x}: {rmse_org}.')
print(f'Miary RMSE dla losowo usuniętych zmiennej {x}: {rmse_SR}.')
print(f'Średnia miary RMSE dla losowo usuniętych zmiennej {x}: {mean_SR:.2f}.')
print(f'Odchylenie standardowe miary RMSE dla losowo usuniętych zmiennej {x}: {std_SR:.2f}.')
print()

x = 'it_quantity'
print(f'Miara RMSE dla oryginalnych wartości zmiennej {x}: {rmse_org_IQ}.')
print(f'Miary RMSE dla losowo usuniętych zmiennej {x}: {rmse_IQ}.')
print(f'Średnia miary RMSE dla losowo usuniętych zmiennej {x}: {mean_IQ:.2f}.')
print(f'Odchylenie standardowe miary RMSE dla losowo usuniętych zmiennej {x}: {std_IQ:.2f}.')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))

ax1.plot(rmse_SR, label = 'RMSE po usunięciu danych')
ax1.plot(range(10), [rmse_org]*10, label = 'RMSE bez usunięcia danych')
ax1.plot(range(10), [mean_SR]*10, label = 'Średnia RMSE po usunięciu danych')
ax1.set_title('Wizualizacji wartości miary RMSE dla zmiennej it_seller_rating')
ax1.set_ylim([34200, 35700])
ax1.legend()
legend = ax1.legend(loc='center right', shadow=True, fontsize='medium')

ax2.plot(rmse_IQ, label = 'RMSE po usunięciu danych')
ax2.plot(range(10), [rmse_org_IQ]*10, label = 'RMSE bez usunięcia danych')
ax2.plot(range(10), [mean_IQ]*10, label = 'Średnia RMSE po usunięciu danych')
ax2.set_title('Wizualizacji wartości miary RMSE dla zmiennej dla zmiennej it_quantity')
ax2.set_ylim([26800, 27700])
ax2.legend()
legend = ax2.legend(loc='center right', shadow=True, fontsize='medium')

plt.show()

Widzimy, że usunięcie wartości zmniejsza wartość miary RMSE dla obu zmiennych. W drugim przypadku, po usunięciu danych z dwóch kolumn wartość miary RMSE dla zmiennej *it_seller_rating* była niższa.